In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read().split('\n')

In [154]:
import numpy as np
import networkx as nx
import sys
sys.path.insert(0, '../')
from aoc_utils import get_grid_neighbours

### Valley plotting functions

In [143]:
def get_tile_to_plot(tile_list):
    if len(tile_list) == 0:
        return '.'
    elif len(tile_list) == 1:
        return tile_list[0]
    else:
        return f'{len(tile_list)}'

def plot_valley_field(valley_field):
    for row in valley_field:
        print(''.join(get_tile_to_plot(tile_list) for tile_list in row))

### Propagating blizzards

In [185]:
wind_map = {'<': 'w', '^': 'n', '>': 'e', 'v': 's'}
def get_neighbour(point, direction):
    neis_4 = get_grid_neighbours(point, 4, as_dict=True)
    nei = neis_4[wind_map[direction]]
    return (nei[0] % M, nei[1] % N)

def propagate_blizzards(valley_field):    
    next_valley_field = [[[] for _ in range(N)] for _ in range(M)]
    for i in range(M):
        for j in range(N):
            for blizzard in valley_field[i][j]:
                next_i, next_j = get_neighbour((i,j), blizzard)
                next_valley_field[next_i][next_j].append(blizzard)
    return next_valley_field

# Part 1

In [260]:
valley = get_input('input') 
M, N = len(valley)-2, len(valley[0])-2
valley_field = [[[t] if (t != '.') else [] for t in list(row[1:-1])] for row in valley[1:-1]]
end_node = (M, N-1)

In [261]:
vf = valley_field.copy()
valley_at_time_t = [vf]
while True:
    vf = propagate_blizzards(vf)
    if vf == valley_field:
        break
    valley_at_time_t.append(vf)

In [262]:
g = nx.DiGraph()
g.add_node((-1, 0, 0))
time = list(range(len(valley_at_time_t)))

for t0, t1, current_valley, next_valley in zip(time, time[1:] + [time[0]], valley_at_time_t, valley_at_time_t[1:] + [valley_at_time_t[0]]):
    # handle start node
    
    # waiting at the start node is always allowed
    g.add_edge((-1, 0, t0), (-1, 0, t1))
    if not next_valley[0][0]:
        # we can move out if there will be no blizzards there
        g.add_edge((-1, 0, t0), (0, 0, t1))
    
    for i in range(M):
        for j in range(N):
            # we only need to consider tiles if they have no blizzards, since we cannot have ended up there otherwise
            if not current_valley[i][j]:
                
                if not next_valley[i][j]:
                    g.add_edge((i,j,t0), (i,j,t1))
                    
                for nei in get_grid_neighbours((i,j), 4):
                    if ((ni := nei[0]) < 0) or ((nj := nei[1]) < 0) or (nei[0] > M - 1) or (nei[1] > N - 1):
                        continue
                    if not next_valley[ni][nj]:
                        g.add_edge((i,j,t0), (ni,nj,t1))
    
    # handle end node
    if not current_valley[M-1][N-1]:
        g.add_edge((M-1, N-1, t0), end_node)

In [265]:
nx.dijkstra_path_length(g, (-1, 0, 0), end_node)

253

# Part 2

In [266]:
for t in range(len(time)):
    # we're always allowed to move to start node
    g.add_edge((0, 0, t), (-1, 0))
    t1 = (t+1)%len(time)
    
    # waiting at the end node is always allowed
    g.add_edge((*end_node, t), (*end_node, t1))
    
    # we can move back out from the end node if there are no blizzards there
    if not valley_at_time_t[t1][M-1][N-1]:
        g.add_edge((*end_node, t), (M-1, N-1, t1))

In [267]:
time_to_end = nx.dijkstra_path_length(g, (-1, 0, 0), end_node)
time_back = nx.dijkstra_path_length(g, (*end_node, time_to_end % len(time)), (-1, 0))
time_to_end_again = nx.dijkstra_path_length(g, (-1, 0, (time_to_end+time_back)%len(time)), end_node)

In [268]:
print(time_to_end + time_back + time_to_end_again)

794
